# Current vs Density 

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import jsonpickle
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st

from tqdm import tqdm_notebook

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
mpl.rc('text', usetex=True)
mpl.rcParams['figure.dpi'] = 150

In [3]:
directory = "D:/DensityDependence2/"

In [4]:
h = 3.9 * ureg.um
tilt_angle = 7*ureg.deg
frequency = 3*ureg.Hz

In [5]:
packing = np.linspace(0.2, 0.9, 51)
packing

array([0.2  , 0.214, 0.228, 0.242, 0.256, 0.27 , 0.284, 0.298, 0.312,
       0.326, 0.34 , 0.354, 0.368, 0.382, 0.396, 0.41 , 0.424, 0.438,
       0.452, 0.466, 0.48 , 0.494, 0.508, 0.522, 0.536, 0.55 , 0.564,
       0.578, 0.592, 0.606, 0.62 , 0.634, 0.648, 0.662, 0.676, 0.69 ,
       0.704, 0.718, 0.732, 0.746, 0.76 , 0.774, 0.788, 0.802, 0.816,
       0.83 , 0.844, 0.858, 0.872, 0.886, 0.9  ])

In [6]:
def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

In [7]:
index_name = "index.dat"

import csv

def prepare_simulation(packing):
    
    region, positions = mgc.hexagonal_order(
        n_of_particles = 250,
        packing=packing, height = h.magnitude,
        r = 1.4, theta = np.pi/2)

    particles = mgc.particles(
        positions*ureg.um,
        radius = 1.4*ureg.um,
        susceptibility = 0.4,
        diffusion=0.073*ureg.um**2/ureg.s,
        density = 1000*ureg.kg/ureg.m**3,
        temperature=300*ureg.K)

    world = mgc.world(particles, temperature = 300*ureg.K,
                      lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)],
                      region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                      dipole_cutoff = 30*ureg.um)

    field = mgc.field(magnitude = 7.28*ureg.mT, frequency = frequency,
                  angle = 27*ureg.degrees, multibody_iter=2)
    
    field = redefine_field(field,tilt_angle)

    sim = mgc.sim(dir_name = directory,
                  file_name="Phi_%s"%(packing),stamp_time=False,
                  timestep = 1e-4*ureg.s, framerate = 75*ureg.Hz, total_time = 60*ureg.s,
                  particles = particles, world = world, field = field, output=["x","y","z","mux","muy","muz","fx","fy"])
    
    sim.packing = packing
    
    return sim

In [8]:
simulations = [prepare_simulation(phi) for phi in packing]

In [9]:
sim = simulations[0]

In [10]:
def run_simulation(sim):
    
    import csv
    import os
    index_name = "index.dat"
    directory = sim.dir_name
    
    sim.generate_scripts()
    sim.run()
    
    packing = sim.packing
    
    with open(os.path.join(directory,index_name),'a',newline='') as file:
        writer = csv.writer(file,delimiter=' ')
        writer.writerow([os.path.split(sim.base_name)[-1], packing])

## Run asynchronously in several cores at once.

https://gist.github.com/aortiza/dd1eb47fa675c147bd44df3a98bc7a4e

In [11]:
from multiprocessing import Pool
import inspect

In [12]:
with open("dummy.py",'w') as file:
    file.write(inspect.getsource(run_simulation))

In [13]:
import dummy
if __name__ ==  '__main__': 
    num_processors = 8
    p=Pool(processes = num_processors)
    p.map(dummy.run_simulation,simulations)